In [2]:
%pylab inline
import cosima_cookbook as cc
import numpy as np
import matplotlib.pyplot as plt # to generate plots
import xarray as xr
import scipy as sp
import pandas as pd
import IPython.display
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap # plot on map projections
from glob import glob
import scipy.interpolate
import scipy.ndimage
from tqdm import tqdm_notebook
import pylab
from matplotlib import ticker, cm
# for georefferenced data
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm # for colormap
from sklearn import preprocessing
import netCDF4 as nc
import dask.array as da
import dataset

Populating the interactive namespace from numpy and matplotlib


In [3]:
import dask
from dask.distributed import Client, progress
from dask_jobqueue import PBSCluster

In [4]:
import socket
import fcntl
import struct
def get_interface_ip(ifname):
   s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
   return socket.inet_ntoa(fcntl.ioctl(
           s.fileno(),
           0x8915,
           struct.pack('256s', bytes(ifname[:15], 'utf-8'))
       )[20:24])

In [4]:
#cluster = Client()
#cluster

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:44243 Dashboard: http://127.0.0.1:36757/status,Cluster Workers: 8 Cores: 32 Memory: 2.15 GB


In [5]:
cluster = PBSCluster(cores=8, memory='20 GB', ip='',interface='ib0',localdirectory='/g/data/e14/ss2778/tmp')
cluster

In [6]:
client = Client(cluster)
client

Client Scheduler: tcp://192.43.239.22:43242 Dashboard: http://192.43.239.22:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [7]:
cluster.job_header = '#!/usr/bin/env bash\n#PBS -N dask-worker\n#PBS -q expressbw\n#PBS -l ncpus=8\n#PBS -l mem=20GB\n#PBS -l walltime=00:30:00\nJOB_ID=${PBS_JOBID%.*}'

In [8]:
cluster.scale(10)

In [9]:
print(cluster.job_script())

#!/usr/bin/env bash

#!/usr/bin/env bash
#PBS -N dask-worker
#PBS -q expressbw
#PBS -l ncpus=8
#PBS -l mem=20GB
#PBS -l walltime=00:30:00
JOB_ID=${PBS_JOBID%.*}



/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/bin/python -m distributed.cli.dask_worker tcp://192.43.239.22:43242 --nthreads 8 --memory-limit 20.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60 --interface ib0



In [10]:
#Creating my own database:
db = '/g/data/e14/ss2778/cc_test1.db'
session = cc.database.create_session(db)

In [11]:
from joblib import Memory
memory = Memory(cachedir='/g/data/e14/ss2778/tmp/',verbose=0)

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/g/data/e14/ss2778/tmp/'", use "location='/g/data/e14/ss2778/tmp/'" instead.
  


In [12]:
dir_list=['/g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf9091']
cc.database.build_index(dir_list,session,update=True)

Indexing experiment: 01deg_jra55v13_ryf9091


0it [00:00, ?it/s]


0

In [ ]:
%time
expt =  '01deg_jra55v13_ryf9091'
var1 = 'uocn_m'
var2 = 'vocn_m'
uocn_m = cc.querying.getvar(expt, var1, session,start_time='1900-01-01',
                            end_time='1920-12-31' )
vocn_m = cc.querying.getvar(expt, var1, session,start_time='1900-01-01',
                            end_time='1920-12-31')

CPU times: user 0 ns, sys: 5 µs, total: 5 µs
Wall time: 9.78 µs


In [7]:
@memory.cache
def calc_eke_dask(expt):

    u = da.from_array(uocn_m,chunks=(60,675,900)) 
    #u = da.concatenate(u_dataarrays, axis=0)

    v = da.from_array(vocn_m,chunks=(60,675,900)) 
    #v = da.concatenate(v_dataarrays, axis=0)

    u_mean = u.mean(axis=0)
    v_mean = v.mean(axis=0)

    u_ = u - u_mean
    v_ = v - v_mean

    EKE = 0.5 * (u_**2 + v_**2)
    EKE_mean = EKE.mean(axis=0)
    
  #  EKE = cc.compute_by_block(EKE)
    
    EKE=EKE.compute()
    return EKE

In [ ]:
%%time
EKE = calc_eke_dask('01deg_jra55v13_ryf9091')

# Other_Method to load u,v:

In [ ]:
#SH = (xr.open_mfdataset('/g/data1/ua8/JRA55-do/RYF/v1-3/RYF.q_10.1990_1991.nc'))
O1 = (xr.open_mfdataset('/g/data/hh5/tmp/cosima/access-om2-01/01deg_jra55v13_ryf9091/output*/ice/OUTPUT/iceh.*.nc'))

In [6]:
u=O1['uocn_m']
v=O1['vocn_m']

In [7]:
u_mean = u.mean('time')
v_mean = v.mean('time')

In [8]:
u_ = u1 - u_mean
v_ = v1 - v_mean

In [ ]:
EKE = 0.5 * (u_**2 + v_**2)

In [ ]:
#EKE_mean_10yeardata
eke10=EKE.isel(time=slice(0,120))
eke10m=eke10.mean('time')
eke10m.plot()
savefigure('mean_eke_10years')

In [ ]:
#Hotspot of EKEmean_10years_data_timeseries
h1=eke10.sel(xu_ocean=slice(,)).sel(yu_ocean=slice(,)) #hotspot 
h2= eke10.sel(xu_ocean=slice(,)).sel(yu_ocean=slice(,)) #hotspot
h3= eke10.sel(xu_ocean=slice(,)).sel(yu_ocean=slice(,)) #hotspot

In [28]:
#Seasonal Anomalies of EKE:
EKE_season_mean = EKE.groupby('time.season').mean('time')
EKE_season = EKE.groupby('time.season')
EKE_season_anom = EKE_season - EKE_season_mean
#area averaged eke anomalies:
de=EKE_season_anom.mean('xt_ocean').mean('yt_ocean')

In [ ]:
#Extracting season from time series:
%%time
for season, values in data.groupby('season'):
    ds=values
    print(season)
    print(values)

In [ ]:
#Plotting different season time series for 30 years:

In [ ]:
plt.figure(figsize=(15,8))
EKE_mean=EKE.mean('time')
X,Y=np.meshgrid(EKE.xu_ocean,EKE.yu_ocean)
h=plt.contourf(X,Y,EKE_mean,levels=100)
cb = plt.colorbar(h,orientation='vertical')
plt.contour(X,Y,EKE,levels=14, linewidths=0.5, colors='k')
plt.title('EKE (m2/s2)')
plt.ylabel('latitude(degrees)')
plt.xlabel('longitude(degrees)')
savefigure('EKE_mean')

In [ ]:
#EKE trend over 1958-2018
eke_time = EKE.time.values.astype('datetime64[M]').astype('float64')
etime = (eke_time-eke_time.mean())/eke_time.std()
eke=EKE-EKE.mean('time')
tl=len(EKE.time)
trend_eke= np.dot(etime,np.transpose(EKE.values,(1,0,2)))/tl

In [ ]:
plt.figure(figsize=(15,8))
X,Y=np.meshgrid(EKE.xu_ocean,EKE.yu_ocean)
h=plt.contourf(X,Y,trend_eke,levels=100)
cb = plt.colorbar(h,orientation='vertical')
plt.contour(X,Y,EKE,levels=14, linewidths=0.5, colors='k')
plt.title('EKE (m2/s2)')
plt.ylabel('latitude(degrees)')
plt.xlabel('longitude(degrees)')
savefigure('Trend_EKE_50years')